In [1]:
from flask import Flask, render_template, request
import psycopg
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [2]:
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="311_service_data",
    user="postgres",
    password="123")

In [3]:
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def home():
    cursor = conn.cursor()
    cursor.execute("""SELECT DISTINCT "Agency" FROM main_request""")
    agencies = cursor.fetchall()
    cursor.execute("""SELECT DISTINCT "Complaint Type" FROM main_request""")
    complaint_types = cursor.fetchall()
    
    options_agency = [{'label': agency[0], 'value': agency[0]} for agency in agencies]
    options_complaint = [{'label': complaint_type[0], 'value': complaint_type[0]} for complaint_type in complaint_types]
    
    if request.method == 'POST':
        agency = request.form['agency']
        complaint_type = request.form['complaint_type']
        zip_code = request.form['zip_code']
        created_date = request.form['created_date']
        
        query = f"""SELECT 
                    main_request."Unique Key"
                    , main_request."Agency"
                    , main_request."Complaint Type"
                    , main_request."Status"
                    , main_request."DaystoClose"
                    , main_request."Closed Date"
                    , main_request."Created Date"
                    , location."Incident Address"
                    , location."Latitude"
                    , location."Longitude"
                    FROM main_request
                    INNER JOIN location ON main_request."location_id" = location."LocID"
                    WHERE main_request."Created Date" >= '{created_date}'::date - interval '30 days'
                    AND location."Incident Zip" = '{zip_code}'
                    """
        if agency:
            query += f"""AND main_request."Agency" = '{agency}'
                        """
        if complaint_type:
            query += f"""AND main_request."Complaint Type" = '{complaint_type}'
                        """
                        
        df = pd.read_sql_query(query, conn)
        
        color_scale = px.colors.qualitative.Prism
        unique_complaint_types = df['Complaint Type'].unique()
        complaint_color_map = {complaint_type: color_scale[i % len(color_scale)] for i, complaint_type in enumerate(unique_complaint_types)}

        complaint_counts = df['Complaint Type'].value_counts().reset_index()
        complaint_counts.columns = ['Complaint Type', 'Count']

        closed_cases = df[df["Status"].isin(['closed'])]
        closed_percent = (closed_cases.groupby("Complaint Type").size() / df.groupby("Complaint Type").size() * 100).reset_index(name="% of Resolved Requests")
        closed_percent["% of Resolved Requests"] = closed_percent["% of Resolved Requests"].round(2)
        avg_closing_time = closed_cases[closed_cases["DaystoClose"] > 0].groupby("Complaint Type")["DaystoClose"].mean().reset_index(name="Average Resolution Days")
        avg_closing_time["Average Resolution Days"] = avg_closing_time["Average Resolution Days"].round(2)

        bar_chart_data = pd.merge(pd.merge(complaint_counts, closed_percent, on="Complaint Type", how="left"), avg_closing_time, on="Complaint Type", how="left").fillna(-1).reset_index(drop=True)
        bar_chart_data = bar_chart_data.sort_values("Count", ascending=False).head(10)
        
        bar_chart = px.bar(bar_chart_data, x="Count", y="Complaint Type", orientation="h", text="Count", hover_data={"% of Resolved Requests": True, "Average Resolution Days": True, "Complaint Type": False, "Count": False}, title="Top of Service Requests by Type", color="Complaint Type", color_discrete_map=complaint_color_map)
        bar_chart.update_traces(texttemplate='%{text:.2s}', textposition='outside')
        bar_chart.update_layout(margin=dict(t=90, l=5, b=0, r=5),showlegend=False)

        map_chart = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", hover_data={"Incident Address": True, "Complaint Type": True, "Longitude": False, "Latitude": False}, title="Map of Service Requests", color="Complaint Type", color_discrete_map=complaint_color_map, zoom=14, opacity=0.5)
        map_chart.update_layout(margin=dict(t=30, l=5, b=0, r=5), mapbox_style="carto-positron")
            
        graph = bar_chart.to_html(full_html=False)
        graph2 = map_chart.to_html(full_html=False)

        return render_template('index.html', options_agency=options_agency, options_complaint=options_complaint, graph=graph, graph2=graph2)

    return render_template('index.html', options_agency=options_agency, options_complaint=options_complaint, graph=None, graph2=None)

In [ ]:
if __name__ == '__main__':
    app.run(host='localhost', port=5004)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5004
Press CTRL+C to quit
127.0.0.1 - - [24/Apr/2023 13:13:50] "GET / HTTP/1.1" 200 -
/var/folders/17/ln01f0qd6mj9c4r0y8js83180000gn/T/ipykernel_35742/1090415611.py:43: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

127.0.0.1 - - [24/Apr/2023 13:18:53] "POST / HTTP/1.1" 200 -
/var/folders/17/ln01f0qd6mj9c4r0y8js83180000gn/T/ipykernel_35742/1090415611.py:43: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

127.0.0.1 - - [24/Apr/2023 13:19:05] "POST / HTTP/1.1" 200 -
